In [1]:
"""
Units Used 
energy: TWh
emissions: gigatons of CO2 equivalent
importance: non-dimensional
"""
emissions2energy = 4.1 # TWh/megatons CO2eq

# Graph node for solutions (green trapezoids)
class Solution:
    def __init__(self, name):
        self.obj = self
        self.name = name
        self.edges_in = []
        self.edges_out = []
    
    def __repr__(self): 
        return self.name 
        
    def add_edge(self,edge):
        if type(edge) == ProbToSol:
            self.edges_in.append(edge)
        elif type(edge) == SolToProb:
            self.edges_out.append(edge)
        else:
            print("Edge object incompatible with this solution node")
    
    # Does this node convert input emissions to energy (true) or propogate the emissions unit (false)
    def convert_emissions(self):
        out = True
        for i in self.edges_out:
            if i.unit == "emissions":
                out = False
        return out
    
    # Returns total energy is energy into the node including converted emissions if applicable
    def energy_in(self):
        total = 0
        for i in [a for a in self.edges_in if a.unit == "energy"]:
            total += i.get_val()
        if self.convert_emissions():
            for i in [a for a in self.edges_in if a.unit == "emissions"]:
                total += emissions2energy*i.get_val()
        return total
    
    # Returns total importance is all of the inputs added
    def importance_in(self):
        total = 0
        for i in self.edges_in:
            if i.unit == "emissions":
                total += emissions2energy*i.get_val()
            else:
                total += i.get_val()
        return total
        
    # Returns total emissions is the sum of emissions inputs        
    def emissions_in(self):
        total = 0
        for i in [a for a in self.edges_in if a.unit == "emissions"]:
            total += i.value
        return total
    
    # Returns sum of feasibilities for all outgoing edges with this unit
    def total_feasibility(self, unit):
        total = 0
        for i in [a for a in self.edges_out if a.unit == unit]:
            total += i.feasibility
        return total

    # Updates output edges based on input edges, returns 1 if node converged
    def assign_output_edges(self):
        changes = 0 # how many edges change values
        # Output edges assigned based on feasibility values
        
        # Assign Emissions Outputs
        c_total = self.emissions_in()
        c_feas = self.total_feasibility("emissions")
        for i in [a for a in self.edges_out if a.unit == "emissions"]:
            changes += i.set_val( i.scalar*( c_total*(i.feasibility/c_feas) ) )
        
        #Assign Importance Outputs
        i_total = self.importance_in()
        i_feas = self.total_feasibility("importance")
        for i in [a for a in self.edges_out if a.unit == "importance"]:
            changes += i.set_val( i.scalar*( i_total*(i.feasibility/i_feas) ) )
        
        #Assign Energy Outputs
        e_total = self.energy_in()
        e_feas = self.total_feasibility("energy")
        for i in [a for a in self.edges_out if a.unit == "energy"]:
            changes += i.set_val( i.scalar*( e_total*(i.feasibility/e_feas) ) )
        
        # Exit flag
        return changes
        
        
        
# Graph node for problems (orange rectangles) 
class Problem:
    def __init__(self, name):
        self.obj = self
        self.name = name
        self.edges_in = []
        self.edges_out = []
        self.emission_sources = []
    
    def __repr__(self): 
        return self.name 
        
    def add_edge(self,edge):
        if type(edge) == ProbToSol:
            self.edges_out.append(edge)
        elif type(edge) == SolToProb:
            self.edges_in.append(edge)
        elif type(edge) == EmissionsSource:
            self.emission_sources.append(edge)
        else:
            print("Edge object incompatible with this problem node")
    
    # Does this node convert input emissions to energy (true) or propogate the emissions unit (false)
    def convert_emissions(self):
        out = True
        for i in self.edges_out:
            if i.unit == "emissions":
                out = False
        return out
    
    # Returns total energy is energy into the node including converted emissions if applicable
    def energy_in(self):
        total = 0
        for i in [a for a in self.edges_in if a.unit == "energy"]:
            total += i.get_val()
        if self.convert_emissions():
            for i in [a for a in self.edges_in if a.unit == "emissions"]:
                total += emissions2energy*i.get_val()
            for i in self.emission_sources:
                total += i.energy
        return total
    
    # Returns total importance is all of the inputs added
    def importance_in(self):
        total = 0
        for i in self.edges_in:
            if i.unit == "emissions":
                total += emissions2energy*i.get_val()
            else:
                total += i.get_val()
        for i in self.emission_sources:
                total += i.energy
        return total
        
    # Returns total emissions is the sum of emissions inputs        
    def emissions_in(self):
        total = 0
        for i in [a for a in self.edges_in if a.unit == "emissions"]:
            total += i.value
        for i in self.emission_sources:
                total += i.emissions
        return total
    
    # Returns sum of feasibilities for all outgoing edges with this unit
    def total_feasibility(self, unit):
        total = 0
        for i in [a for a in self.edges_out if a.unit == unit]:
            total += i.feasibility
        return total

    # Updates output edges based on input edges, returns 1 if node converged
    def assign_output_edges(self):
        changes = 0 # how many edges change values
        # Output edges assigned based on feasibility values
        
        # Assign Emissions Outputs
        c_total = self.emissions_in()
        c_feas = self.total_feasibility("emissions")
        for i in [a for a in self.edges_out if a.unit == "emissions"]:
            changes += i.set_val( i.scalar*( c_total*(i.feasibility/c_feas) ) )
        
        #Assign Importance Outputs
        i_total = self.importance_in()
        i_feas = self.total_feasibility("importance")
        for i in [a for a in self.edges_out if a.unit == "importance"]:
            changes += i.set_val( i.scalar*( i_total*(i.feasibility/i_feas) ) )
        
        #Assign Energy Outputs
        e_total = self.energy_in()
        e_feas = self.total_feasibility("energy")
        for i in [a for a in self.edges_out if a.unit == "energy"]:
            changes += i.set_val( i.scalar*( e_total*(i.feasibility/e_feas) ) )

        # Exit flag
        return changes
        

# Graph edge representing the amount of emissions caused by a given problem
class EmissionsSource:
    def __init__(self, name, emissions, energy = None):
        self.obj = self
        self.name = name
        self.emissions = emissions
        if energy == None:
            self.energy = emissions2energy*emissions # conversion is average MWh/t CO2
        else:
            self.energy = energy
        
    def __repr__(self): 
        return self.name 
    
    def connect(self, obj):
        obj.add_edge(self)

    
# Graph edge for solutions to problems (green arrow)
class ProbToSol:
    def __init__(self, name, scalar = 1, feasibility = 1, unit = "energy", connect = None):
        self.obj = self
        self.name = name
        self.scalar = scalar
        self.unit = unit
        self.feasibility = feasibility
        self.value = None
        if connect != None:
            self.connect(connect[0],connect[1])
        
    def __repr__(self): 
        return self.name 
    
    def get_val(self):
        if self.value == None:
            print("Edge " + self.name + " has no assigned value and is returning zero")
            return 0
        return self.value
    
    def set_val(self,val):
        if val != self.value:
            self.value = val
            return 1
        return 0
    
    # Connects this edge to the two nodes "obj1" and "obj2"
    def connect(self, obj1, obj2):
        obj1.add_edge(self)
        obj2.add_edge(self)

    
# Graph Edge for problems that arise for a solution (red arrow)
class SolToProb:
    def __init__(self, name, scalar = 1, feasibility = 1, unit = "energy", connect = None):
        self.obj = self
        self.name = name
        self.scalar = scalar 
        self.unit = unit
        self.feasibility = feasibility
        self.value = None
        if connect != None:
            self.connect(connect[0],connect[1])
    
    def __repr__(self): 
        return self.name 
    
    def get_val(self):
        if self.value == None:
            print("Edge " + self.name + " has no assigned value and is returning zero")
            return 0
        return self.value
    
    def set_val(self,val):
        if val != self.value:
            self.value = val
            return 1
        return 0
    
    # Connects this edge to the two nodes "obj1" and "obj2"
    def connect(self, obj1, obj2):
        obj1.add_edge(self)
        obj2.add_edge(self)

In [2]:
class EmissionsGraph(EmissionsSource):

    def em_connect(self, obj1, obj2, weight):
        self.em_nodes.add(obj1)
        self.em_nodes.add(obj2)
        self.em_edges.append([obj1, obj2, weight])
    

In [3]:
class ClimateGraph(EmissionsGraph):
    
    def __init__(self):
        self.nodes = []
        self.edges = []
        self.sources = []
        self.max_it = 25
        self.em_nodes = set()
        self.em_edges = []
    
    def Solution(self, name):
        obj = Solution(name)
        self.nodes.append(obj)
        return obj
        
    def Problem(self, name):
        obj = Problem(name)
        self.nodes.append(obj)
        return obj
        
    def SolToProb(self,name, scalar = 1, feasibility = 1, unit = "energy", connect = None):
        if unit not in {"energy", "importance", "emissions"}:
            print("Invalid unit")
        obj = SolToProb(name, scalar, feasibility, unit, connect)
        self.edges.append(obj)
        return obj
        
    def ProbToSol(self,name, scalar = 1, feasibility = 1, unit = "energy", connect = None):
        if unit not in {"energy", "importance", "emissions"}:
            print("Invalid unit")
        obj = ProbToSol(name, scalar, feasibility, unit, connect)
        self.edges.append(obj)
        return obj
        
    def EmissionsSource(self, name, emissions, energy = None):
        obj = EmissionsSource(name, emissions, energy)
        self.sources.append(obj)
        return obj
    
    
    # Finds problem nodes with emission sources attached
    def start_nodes(self):
        start_nodes = set()
        for n in self.nodes:
            if type(n)==Problem:
                if len(n.emission_sources)>0:
                    start_nodes.add(n)
        return start_nodes
    
    # Returns dictionary where an edge key returns a list with the [output node, input node]
    def find_connections(self):
        connections = dict()
        for n in self.nodes:
            for e in n.edges_out:
                connections[e] = [n]
        for n in self.nodes:
            for e in n.edges_in:
                connections[e].append(n)
        for c in connections.values():
            if len(c)!=2:
                print("Error: Some edges not connected to exactly 2 nodes")
        self.connections = connections
        
    def converge_graph(self):
        self.find_connections() # Lookup table for edges
        
        changes = 1
        it = 0
        # Iterate until changes stop or hits max_it
        while changes > 0 and it < self.max_it:
            it +=1
            changes = 0
            
            # One full interation of the graph
            current_nodes = self.start_nodes() # Start search with starting nodes
            visited_nodes = set()
            while len(current_nodes)>0:
                
                next_nodes = set()
                # For each node in current_nodes, update output edges
                for n in current_nodes:
                    visited_nodes.add(n)
                    changes +=  n.assign_output_edges()
                    # Make list of connected nodes that haven't been visited yet
                    for e in n.edges_out:
                        n_candidate = self.connections[e][1] # find node who has that edges as a candidate
                        if n_candidate not in visited_nodes:
                            next_nodes.add(n_candidate)
                current_nodes = next_nodes
                #print("Visited nodes: " + str(visited_nodes))
                print("Next nodes: " + str(next_nodes))
            print("Changes in this iteration: " + str(changes))
        print("Converged with " + str(it) + " iterations.")

In [4]:
g = ClimateGraph()
total_emissions = 49.4 # in gigatons of CO2 eq (2016, ourworldindata.org/emissions-by-sector)

### EMISSIONS ###
#(2016, ourworldindata.org/emissions-by-sector)

#Direct Fuel
E_steel = g.EmissionsSource("Iron and Steel GHGs", 0.072*total_emissions)

E_car = g.EmissionsSource("Cars GHGs", 0.0817*total_emissions)

E_plane = g.EmissionsSource("Aviation GHGs", 0.019*total_emissions)

E_ship = g.EmissionsSource("Shipping GHGs", 0.024*total_emissions)

E_truck_machine = g.EmissionsSource("Trucks and Machines GHGs", 0.0793*total_emissions)
E_truck = g.EmissionsSource("Trucks GHGs", 0.0373*total_emissions)
E_ag = g.EmissionsSource("Fishing and Farming GHGs", 0.017*total_emissions) # goes to heavy machinery
E_machines = g.EmissionsSource("Machines in Industry GHGs", 0.035*total_emissions) # Total guess, 25% of non-building energy in industry
g.em_connect(E_truck_machine, E_truck, E_truck.emissions)
g.em_connect(E_truck_machine, E_ag, E_ag.emissions)
g.em_connect(E_truck_machine, E_machines, E_machines.emissions)

E_ammonia = g.EmissionsSource("Ammonia GHGs", 0.013*total_emissions)

E_misc_burn = g.EmissionsSource("Misc Fuel Combustion GHGs", 0.078*total_emissions) # electricity and heat from biomass; on-site heat sources; combined heat and power (CHP)

E_direct_fuel = g.EmissionsSource("Direct Fuel Use GHGs", (0.352+0.0315)*total_emissions) # adding in fugitive emissions
g.em_connect(E_direct_fuel, E_steel, E_steel.emissions)
g.em_connect(E_direct_fuel, E_car, E_car.emissions)
g.em_connect(E_direct_fuel, E_plane, E_plane.emissions)
g.em_connect(E_direct_fuel, E_ship, E_ship.emissions)
g.em_connect(E_direct_fuel, E_truck, E_truck.emissions)
g.em_connect(E_direct_fuel, E_ag, E_ag.emissions)
g.em_connect(E_direct_fuel, E_machines, E_machines.emissions)
g.em_connect(E_direct_fuel, E_ammonia, E_ammonia.emissions)
g.em_connect(E_direct_fuel, E_misc_burn, E_misc_burn.emissions)

# Electricity, machines, fuel for heating

# Electricity 
#37.7% commercial: 34% heating, 16% cooling, 10.4% lighting, 5% cooking, 34.5% other (EIA)
#62.3% residential: 62% heating, 8% cooling, 5% lighting, 12% cooking, 12% other (EIA)
E_building = g.EmissionsSource("Buildings GHGs", 0.175*total_emissions) # these two are heating, cooling, lighting, appliances, cooking
E_cooking = g.EmissionsSource("Cooking GHGs", 0.0936*0.175*total_emissions)
E_heating = g.EmissionsSource("Building and Water Heating GHGs", 0.514*0.175*total_emissions)
E_cooling = g.EmissionsSource("Cooling GHGs", 0.11*0.175*total_emissions)
E_lighting = g.EmissionsSource("Lighting GHGs", 0.0703*0.175*total_emissions)
E_other = g.EmissionsSource("Other Loads GHGs", 0.205*0.175*total_emissions)
g.em_connect(E_building, E_cooking, E_cooking.emissions)
g.em_connect(E_building, E_heating, E_heating.emissions)
g.em_connect(E_building, E_cooling, E_cooling.emissions)
g.em_connect(E_building, E_lighting, E_lighting.emissions)
g.em_connect(E_building, E_other, E_other.emissions)

E_industry = g.EmissionsSource("Other Industry Energy GHGs", 0.122*total_emissions) # including metals, paper, mining and quarrying, construction, textiles, wood products, and transport equipment, pharmaceuticals, refrigerants
E_industry_heat = g.EmissionsSource("Industry Heat GHGs", 0.5*0.122*total_emissions) # https://www.eia.gov/outlooks/ieo/pdf/industrial.pdf
E_industry_elec = g.EmissionsSource("Industry Electricity GHGs", 0.5*0.122*total_emissions)
g.em_connect(E_industry, E_industry_heat, E_industry_heat.emissions)
g.em_connect(E_industry, E_industry_elec, E_industry_elec.emissions)

E_elec_heat = g.EmissionsSource("Electricity and Heat GHGs", (0.297+0.0265)*total_emissions) # adding in fugitive emissions
g.em_connect(E_elec_heat, E_building, E_building.emissions)
g.em_connect(E_elec_heat, E_industry, E_industry.emissions)

# Direct Emissions
E_cement = g.EmissionsSource("Cement Direct GHGs", 0.03*total_emissions)
E_waste = g.EmissionsSource("Landfills and Wastewater GHGs", 0.032*total_emissions)
E_livestock = g.EmissionsSource("Livestock and Manure GHGs", 0.058*total_emissions)
E_land = g.EmissionsSource("Soil, Rice, and Cropland GHGs", 0.069*total_emissions)
E_deforest = g.EmissionsSource("Deforestation and Crop Burning GHGs", 0.057*total_emissions)
E_chemical = g.EmissionsSource("Chemical Direct GHGs", 0.022*total_emissions) # ammonia and other direct

E_direct = g.EmissionsSource("Direct Emissions GHGs", 0.268*total_emissions)
g.em_connect(E_direct, E_cement, E_cement.emissions)
g.em_connect(E_direct, E_waste, E_waste.emissions)
g.em_connect(E_direct, E_livestock, E_livestock.emissions)
g.em_connect(E_direct, E_land, E_land.emissions)
g.em_connect(E_direct, E_deforest, E_deforest.emissions)
g.em_connect(E_direct, E_chemical, E_chemical.emissions)

# Fugitive Emissions (5.8%)
E_fugitive_elec = g.EmissionsSource("Electricity Fugitive Emissions GHGs", 0.0265*total_emissions) # from coal, oil, and methane, spread across as a scalar
E_fugitive_fuel = g.EmissionsSource("Fuel Fugitive Emissions GHGs", 0.0315*total_emissions) # from coal, oil, and methane, spread across as a scalar
g.em_connect(E_fugitive_fuel, E_direct_fuel, 0.0315*total_emissions)
g.em_connect(E_fugitive_elec, E_elec_heat, 0.0265*total_emissions)

#g.converge_graph()

In [5]:
### PROBLEMS ###

# Connect Problems to Emissions (Primary Problems)
P_car = g.Problem("Cars")
E_car.connect(P_car)
P_steel = g.Problem("Iron and Steel")
E_steel.connect(P_steel)
P_ship = g.Problem("Shipping")
E_ship.connect(P_ship)
P_ammonia = g.Problem("Ammonia")
E_ammonia.connect(P_ammonia)
P_plane = g.Problem("Aviation")
E_plane.connect(P_plane)
P_misc_burn = g.Problem("Misc Fuels")
E_misc_burn.connect(P_misc_burn)
P_truck_machine = g.Problem("Trucks and Machines")
E_truck_machine.connect(P_truck_machine)
P_livestock = g.Problem("Livestock")
E_livestock.connect(P_livestock)
P_cement = g.Problem("Cement")
E_cement.connect(P_cement)
P_deforest = g.Problem("Deforestation and Crop Burning")
E_deforest.connect(P_deforest)
P_chemical = g.Problem("Chemical Production")
E_chemical.connect(P_chemical)
P_land = g.Problem("Land Use")
E_land.connect(P_land)
P_waste = g.Problem("Waste")
E_waste.connect(P_waste)

# Electricity and Heat Problems
P_electricity = g.Problem("Electricity Generation")
E_industry_elec.connect(P_electricity)
E_lighting.connect(P_electricity)
E_cooling.connect(P_electricity)
E_lighting.connect(P_electricity)
E_other.connect(P_electricity)
P_cooking = g.Problem("Cooking")
E_cooking.connect(P_cooking)
P_heating = g.Problem("Heating")
E_heating.connect(P_heating)
P_industry_heat = g.Problem("Industrial Heat")
E_industry_heat.connect(P_industry_heat)

# Secondary Problems
P_alt_fuel = g.Problem("Alternative Fuel Production")
P_metal_mining = g.Problem("Rare Earth Metal Mining")
P_intermittency = g.Problem("Generation Intermittency")
P_spatial_varied = g.Problem("Spatially Varied Power Resources")
P_gas_emissions = g.Problem("Natural Gas Emissions")
P_carbon_storage = g.Problem("Carbon Storage")
P_h2_production = g.Problem("H2 Production")
P_high_land_use = g.Problem("High Land Use")

In [6]:
### SOLUTIONS ###

# replace solution name, name, unit+=, problem, scalar/feasibility
S_elec_transit = g.Solution("Electric Public Transit")
g.SolToProb("Increased Electricity Demand", scalar=0.1, unit = "energy", connect = [S_elec_transit, P_electricity])
g.SolToProb("Rare Metals for Batteries", scalar=0.7, unit = "importance", connect = [S_elec_transit, P_metal_mining])
g.ProbToSol("Electric Buses and Trains", feasibility=1.0, unit="energy", connect=[S_elec_transit, P_car])

S_ev = g.Solution("Electric Vehicles")
g.SolToProb("Increased Electricity Demand", scalar=0.3, unit = "energy", connect = [S_ev, P_electricity])
g.SolToProb("Rare Metals for Batteries", scalar=0.7, unit = "importance", connect = [S_ev, P_metal_mining])
g.ProbToSol("Electric Cars", feasibility=0.9, unit="energy", connect=[S_ev, P_car])
g.ProbToSol("Electric Trucks and Machines", feasibility=0.2, unit="energy", connect=[S_ev, P_truck_machine])
g.ProbToSol("Demand Response with EV Charging", feasibility=0.5, unit="importance", connect=[S_ev, P_intermittency])


S_h2_vehicle = g.Solution("Hydrogen Vehicles")
g.SolToProb("Increased Hydrogen Demand", scalar=1.0, unit = "energy", connect = [S_h2_vehicle, P_h2_production])
g.ProbToSol("Hydrogen Trucks and Machines", feasibility=0.4, unit="energy", connect=[S_h2_vehicle, P_truck_machine])
g.ProbToSol("Hydrogen Airplanes", feasibility=0.3, unit="energy", connect=[S_h2_vehicle, P_plane])


S_alt_vehicle = g.Solution("Alternativ Fuel Vehicle")
g.SolToProb("Alternative Fuel Demand", scalar=1.0, unit = "energy", connect = [S_alt_vehicle, P_alt_fuel])
g.ProbToSol("Synthetic Fuels for Machines and Trucks", feasibility=0.5, unit="energy", connect=[S_alt_vehicle, P_truck_machine])
g.ProbToSol("Jet Fuel Produced with Hydrogen", feasibility=0.8, unit="energy", connect=[S_alt_vehicle, P_plane])
g.ProbToSol("Synthetic Fuel for Shipping", feasibility=0.6, unit="energy", connect=[S_alt_vehicle, P_ship])


S_ft = g.Solution("Fischer Tropsch Process")
g.SolToProb("Increased Hydrogen Demand", scalar=1.3, unit = "energy", connect = [S_ft, P_h2_production])
g.ProbToSol("Synthetic Fuel using FT Process", feasibility=0.9, unit="energy", connect=[S_ft, P_alt_fuel])


S_therm_storage = g.Solution("Thermal Energy Storage")
g.ProbToSol("Storage for Generation Intermittency", feasibility=0.6, unit="importance", connect=[S_therm_storage, P_intermittency])
g.ProbToSol("Storage for Industrial Heating", feasibility=0.8, unit="importance", connect=[S_therm_storage, P_industry_heat])


S_nuclear = g.Solution("Nuclear")
g.ProbToSol("Nuclear Power Gen", feasibility=1.0, unit="energy", connect=[S_nuclear, P_electricity])
g.ProbToSol("Nuclear Barges", feasibility=0.2, unit="energy", connect=[S_nuclear, P_ship])


S_battery = g.Solution("Chemical Battery")
g.SolToProb("Rare Metals for Batteries", scalar=0.7, unit = "importance", connect = [S_battery, P_metal_mining])
g.ProbToSol("Storage for Generation Intermittency", feasibility=0.8, unit="importance", connect=[S_battery, P_intermittency])


S_wind = g.Solution("Wind Turbines")
g.SolToProb("Generation Intermittency", scalar=1.0, unit = "importance", connect = [S_wind, P_intermittency])
g.SolToProb("Wind Resources Localized", scalar=0.5, unit = "importance", connect = [S_wind, P_spatial_varied])
g.ProbToSol("Wind Power", feasibility=1.0, unit="energy", connect=[S_wind, P_electricity])


S_pump_hydro = g.Solution("Pumped Hydro")
g.ProbToSol("Storage for Generation Intermittency", feasibility=1.0, unit="importance", connect=[S_pump_hydro, P_intermittency])


S_pv = g.Solution("Photovoltaics")
g.SolToProb("Generation Intermittency", scalar=1.0, unit = "importance", connect = [S_pv, P_intermittency])
g.SolToProb("Solar Resources Distributed", scalar=0.5, unit = "importance", connect = [S_pv, P_spatial_varied])
g.ProbToSol("Solar Power", feasibility=0.9, unit="energy", connect=[S_pv, P_electricity])


S_h2_storage = g.Solution("Hydrogen Storage")
g.SolToProb("Need for Hydrogen Production and Infrastructure", scalar=0.5, unit = "importance", connect = [S_h2_storage, P_h2_production])
g.ProbToSol("Storage for Generation Intermittency", feasibility=1.0, unit="importance", connect=[S_h2_storage, P_intermittency])


S_geothermal = g.Solution("Geothermal")
g.ProbToSol("Geothermal Power", feasibility=0.6, unit="energy", connect=[S_geothermal, P_electricity])
g.ProbToSol("Geothermal Heating", feasibility=0.5, unit="energy", connect=[S_geothermal, P_heating])


S_grid_infra = g.Solution("Grid Infrastructure")
g.ProbToSol("Long-Distance Grid Interconnections and DERs", feasibility=0.8, unit="importance", connect=[S_grid_infra, P_spatial_varied])


S_solar_therm = g.Solution("Concentrated Solar Thermal")
g.SolToProb("Solar Thermal Resources Localized", scalar=0.7, unit = "importance", connect = [S_solar_therm, P_spatial_varied])
g.ProbToSol("Solar Power", feasibility=0.7, unit="energy", connect=[S_solar_therm, P_electricity])
g.ProbToSol("Solar Water Splitting", feasibility=0.4, unit="energy", connect=[S_solar_therm, P_h2_production])


# TODO find actual energy/ton CO2 captured
S_air_cc = g.Solution("Direct Air Carbon Capture")
g.SolToProb("Need for Carbon Storage", scalar=1.0, unit = "importance", connect = [S_air_cc, P_carbon_storage])
g.SolToProb("Increased Electricity Demand", scalar=0.1, unit = "energy", connect = [S_air_cc, P_electricity])
g.ProbToSol("Air Travel Carbon Removal", feasibility=0.5, unit="energy", connect=[S_air_cc, P_plane])
g.ProbToSol("Shipping Carbon Removal", feasibility=0.5, unit="energy", connect=[S_air_cc, P_ship])
g.ProbToSol("Land Use Carbon Removal", feasibility=0.5, unit="energy", connect=[S_air_cc, P_land])
g.ProbToSol("Livestock GHG Offsets", feasibility=0.5, unit="energy", connect=[S_air_cc, P_livestock])


S_concentrate_cc = g.Solution("Concentrated Carbon Capture")
g.SolToProb("Need for Carbon Storage", scalar=1.0, unit = "importance", connect = [S_concentrate_cc, P_carbon_storage])
g.SolToProb("Increased Electricity Demand", scalar=0.05, unit = "energy", connect = [S_concentrate_cc, P_electricity])
g.ProbToSol("Natural Gas Carbon Removal", feasibility=0.6, unit="energy", connect=[S_concentrate_cc, P_gas_emissions])
g.ProbToSol("Chemical Production Carbon Removal", feasibility=0.6, unit="energy", connect=[S_concentrate_cc, P_chemical])
g.ProbToSol("Cement Carbon Removal", feasibility=0.6, unit="energy", connect=[S_concentrate_cc, P_cement])


S_gas = g.Solution("Natural Gas")
g.SolToProb("Natural Gas Emissions", scalar=1.0, unit = "energy", connect = [S_gas, P_gas_emissions])
g.ProbToSol("Gas for Industrial Heat", feasibility=0.7, unit="energy", connect=[S_gas, P_industry_heat])
g.ProbToSol("Gas Power", feasibility=0.7, unit="energy", connect=[S_gas, P_electricity])
g.ProbToSol("Blue Hydrogen Production", feasibility=0.7, unit="energy", connect=[S_gas, P_h2_production])


S_cook = g.Solution("Electric Range")
g.SolToProb("Increased Electricity Demand", scalar=0.8, unit = "energy", connect = [S_cook, P_electricity])
g.ProbToSol("Electric Range", feasibility=1.0, unit="energy", connect=[S_cook, P_cooking])


S_electrolyzer = g.Solution("Hydrogen Electrolyzers")
g.SolToProb("Increased Electricity Demand",scalar=1.2,  unit = "energy", connect = [S_electrolyzer, P_electricity])
g.ProbToSol("Green Hydrogen Production", feasibility=0.8, unit="energy", connect=[S_electrolyzer, P_h2_production])


#TODO: FIND fertilizer use
S_biomass = g.Solution("Biomass")
g.SolToProb("Increased Fertilizer Needs", scalar = 0.1, unit = "energy", connect = [S_biomass, P_ammonia])
g.SolToProb("High Land Use", scalar=0.6, unit = "importance", connect = [S_biomass, P_high_land_use])
g.ProbToSol("Biomass Gasification", feasibility=0.5, scalar=1.0, unit="energy", connect=[S_biomass, P_h2_production])


S_heat_pump = g.Solution("Heat Pumps")
g.SolToProb("Increased Electricity Demand", scalar=0.4, unit = "energy", connect = [S_heat_pump, P_electricity])
g.ProbToSol("Replace Boilers With Heat Pumps", feasibility=0.9, unit="energy", connect=[S_heat_pump, P_heating])


S_h2_steel = g.Solution("Pure Hydrogen Steel")
g.SolToProb("Increased Hydrogen Demand",scalar=1.0,  unit = "energy", connect = [S_h2_steel, P_h2_production])
g.ProbToSol("Hydrogen DR + Arc Furnace", feasibility=0.7, unit="energy", connect=[S_h2_steel, P_steel])


S_h2_infra = g.Solution("Hydrogen Infrastructure")
g.ProbToSol("Hydrogen Transportation Infrastructure", feasibility=0.7, unit="importance", connect=[S_h2_infra, P_h2_production])


S_elec_steel = g.Solution("Steel Electrolysis")
g.SolToProb("Increased Electricity Demand", scalar=0.8, unit = "energy", connect = [S_elec_steel, P_electricity])
g.ProbToSol("Steel Electrolysis replacing Coke", feasibility=0.4, scalar=1.0, unit="energy", connect=[S_elec_steel, P_steel])


S_dri_steel = g.Solution("DRI with Methane")
g.SolToProb("Methane Demand", scalar=1.0, unit = "energy", connect = [S_dri_steel, P_gas_emissions])
g.ProbToSol("DRI with Methane", feasibility=0.6, unit="energy", connect=[S_dri_steel, P_steel])


S_heat_fuels = g.Solution("Heating with Hydrogen")
g.SolToProb("Increased Hydrogen Demand", scalar=1.0, unit = "energy", connect = [S_heat_fuels, P_h2_production])
g.ProbToSol("Hydrogen Combustion", feasibility=0.7, unit="energy", connect=[S_heat_fuels, P_industry_heat])


S_heat_elec = g.Solution("Electric Heating")
g.SolToProb("Increased Electricity Demand", scalar=1.0, unit = "energy", connect = [S_heat_elec, P_electricity])
g.ProbToSol("Ohmic Heating", feasibility=1.0, unit="energy", connect=[S_heat_elec, P_industry_heat])


S_electrify = g.Solution("Overall Electrification")
g.SolToProb("Increased Electricity Demand", scalar=0.6, unit = "energy", connect = [S_electrify, P_electricity])
g.ProbToSol("Replace Biomass and Extraneous Burning", feasibility=0.6, unit="energy", connect=[S_electrify, P_misc_burn])

S_hb = g.Solution("Haber-Bosch Process")
g.SolToProb("Increased Hydrogen Demand", scalar=1.0, unit = "energy", connect = [S_hb, P_h2_production])
g.ProbToSol("Ammonia Production with HB Process", feasibility=1.0, unit="energy", connect=[S_hb, P_ammonia])


S_methane_capture = g.Solution("Waste Methane Capture")
g.ProbToSol("Landfill + Wastwater Methane Capture", feasibility=0.6, unit="energy", connect=[S_methane_capture, P_waste])
g.SolToProb("Captured methane generated electricity", scalar = 0.04, unit="energy", connect=[S_methane_capture, P_electricity])


S_policy = g.Solution("Policy + Land Management")
g.ProbToSol("Stop Deforestation and Crop Burning", feasibility=0.5, unit="energy", connect=[S_policy, P_deforest])
g.ProbToSol("Improve Land Use Practices", feasibility=0.7, unit="energy", connect=[S_policy, P_land])


S_veggie = g.Solution("Alternative Protein")
g.ProbToSol("Plant Alternatives, Different Animals, Less Protein", feasibility=0.7, unit="importance", connect=[S_veggie, P_livestock])
g.converge_graph()

# VISUALIZATION TODOs
# publish to website 

# GRAPH TODOs
# energy losses with storage
# less ammonia from reduced meat consumption
# get conversion right with 2nd law generator efficiencies
# have downstream node prices affect upstream split

# LATER TODOs
# scalars and feasibilities not realistically set
# documentation on scalars, emissions, and feasibility
# give it all prices, rank solutions by change in overall price vs delta in feasibility

Edge Increased Fertilizer Needs has no assigned value and is returning zero
Edge Increased Fertilizer Needs has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Increased Electricity Demand has no assigned value and is returning zero
Edge Captured methane generated electricity has no assigned value an

In [7]:
from pyvis.network import Network
import param, panel as pn
from panel import widgets

pn.extension()

# Generate Graphic with Pyviz
node_size_scale = pn.widgets.FloatSlider(name = "Node Distance", start = 10, end = 60, step = 1, value = 30)
edge_width_scale = pn.widgets.FloatSlider(name = "Node Distance", start = 0.2, end = 2, step = 0.1, value = 0.9)
node_dist = pn.widgets.FloatSlider(name = "Node Distance", start = 50, end = 1000, step = 50, value = 300)
emission_show = pn.widgets.Checkbox(name = "Show Sources of Emissions ", value = True)
inline_format = False

def create_graph(g):
    
    net = Network(height='1000px', width='100%', bgcolor='#222222', font_color='white', directed=True, notebook= inline_format)
    net.repulsion(node_distance=node_dist.value, spring_length=int(node_dist.value/5))

   
    if emission_show.value is True:
        # Add Emission Graph
        for n in g.em_nodes:
            title_e = "Emissions: "+str(int(10*n.emissions)/10)+" gigatons CO2"
            n_size = (n.energy*node_size_scale.value)**0.5
            net.add_node(n.name, label= n.name, title = title_e, size = n_size, color = '#e69138')
        for e in g.em_edges:
            title_e = "Emissions: "+str(int(10*e[2])/10)+" gigatons CO2"
            e_width = e[2]*emissions2energy*edge_width_scale.value
            net.add_edge(e[0].name, e[1].name, title = title_e, width = e_width, color = '#e69138')

    # Graph Nodes
    for n in g.nodes:
        node_importance = n.importance_in()
        node_energy = n.energy_in()
        title = "Node Importance: "+str(int(10*node_importance)/10)
        if node_energy > 0:
            title += "  , Node Energy: "+str(int(10*node_energy)/10)+" TWh"

        if type(n)==Solution:
            n_size = (node_importance*node_size_scale.value)**0.5
            net.add_node(n.name, label = n.name, title = title, size = n_size, color = '#5cb85c')
        else:
            n_size = (node_importance*node_size_scale.value)**0.5
            net.add_node(n.name, label = n.name, title = title, size = n_size, color = '#d9534f')

            if emission_show.value is True:
                # Graph Emission Sources connected to nodes
                for i in n.emission_sources:
                    title_e = "Emissions: "+str(int(10*i.emissions)/10)+" gigatons CO2"
                    e_width = i.energy*edge_width_scale.value
                    net.add_edge(i.name, n.name, title = title_e, width = e_width, color = '#e69138')

    # Graph Edges
    for e in g.edges:

        if type(e)==ProbToSol:
            title = str(e.name)+": Value("+e.unit+"): "+str(int(10*e.value)/10)+" , Feasibility: "+str(e.feasibility)
            e_width = e.value*edge_width_scale.value
            net.add_edge(g.connections[e][0].name, g.connections[e][1].name, title = title, width = e_width, color = '#008744', arrowStrikethrough=False)
        else: # SolToProb
            title = str(e.name)+": Value("+e.unit+"): "+str(int(10*e.value)/10)+" , Scalar: "+str(e.scalar)
            e_width = e.value*edge_width_scale.value
            net.add_edge(g.connections[e][0].name, g.connections[e][1].name, title = title, width = e_width, color = '#d62d20', arrowStrikethrough=False)

    net.barnes_hut(damping = 0.9)

    return net.show('ClimateGraph.html')


# CREATE UI FOR GRAPH
remake_graph = pn.widgets.Toggle(name='Generate Graph', button_type='success')
node1_sel = pn.widgets.Select(name = "First Node Select (edge tail)", options = g.nodes, value = g.nodes[0])
node2_sel = pn.widgets.Select(name = "Second Node Select (edge arrow)", options = g.nodes, value = g.nodes[26])
value_sel = pn.widgets.FloatSlider(name = "New Edge Value (feasibility or scalar)", start = 0, end = 2, step = 0.1, value = 0.9)

def plot_g(remake_graph):
    n1 = node1_sel.value
    n2 = node2_sel.value
    d = g.connections
    v = list(d.values())
    k = list(d.keys())
    idx = -1
    print
    for i in range(len(v)):
        if v[i] == [n1,n2]:
            idx = i
    if idx==-1:
        print("Matchinig Edge Not Found for Selected Nodes")
    else:
        if type(k[idx]) == ProbToSol:
            k[idx].feasibility = value_sel.value
        elif type(k[idx]) == SolToProb:
            k[idx].scalar = value_sel.value
        else:
            print("Incompatoble Edge Type")
        
    return create_graph(g)

pn.Row(
    pn.WidgetBox(emission_show, node_size_scale, edge_width_scale, node_dist, node1_sel, node2_sel, value_sel),
    pn.WidgetBox(remake_graph),  
    pn.bind(plot_g, remake_graph)
)



Row
    [0] WidgetBox
        [0] Checkbox(name='Show Sources o..., value=True)
        [1] FloatSlider(end=60, name='Node Distance', start=10, step=1, value=30)
        [2] FloatSlider(end=2, name='Node Distance', start=0.2, value=0.9)
        [3] FloatSlider(end=1000, name='Node Distance', start=50, step=50, value=300)
        [4] Select(name='First Node Select (..., options=[Cars, Iron and Steel, ...], value=Cars)
        [5] Select(name='Second Node S..., options=[Cars, Iron and Steel, ...], value=Electric Vehicles)
        [6] FloatSlider(end=2, name='New Edge Value (..., value=0.9)
    [1] WidgetBox
        [0] Toggle(button_type='success', name='Generate Graph')
    [2] ParamFunction(function)

In [8]:
# EXAMPLE GRAPH: Create the Graph

g = ClimateGraph()
P1 = g.Problem("H2 Production")
S1 = g.Solution("H2 Electrolysis")
S2 = g.Solution("Nuclear")

g.ProbToSol("Electrolyzers", feasibility = 0.9, unit = "energy", connect = [P1, S1])

e2 = g.ProbToSol("Nuclear Hydrogen", feasibility = 0.3, unit = "energy")
e2.connect(P1, S2)
E1 = g.EmissionsSource("Hydrogen Emissions", 10)
E1.connect(P1)
E2 = g.EmissionsSource("Carbon Emissions", 30)
g.em_connect(E1, E2, 10)

e3 = g.SolToProb("Increased Electricity Demand", scalar = 0.8)
P2 = g.Problem("Electricity Generation")
e3.connect(P2,S1)
S3 = g.Solution("H2 Fuel Cells")
e4 = g.ProbToSol("H2 Fuel Cells Produce Electricity", unit = "importance")
e4.connect(S3,P2)
e5 = g.SolToProb("Increased Hydrogen Demand", scalar = 1.2, unit = "importance")
e5.connect(S3,P1)


# Solve for the graph numerically
g.converge_graph()


Edge Increased Hydrogen Demand has no assigned value and is returning zero
Next nodes: {H2 Electrolysis, Nuclear}
Next nodes: {Electricity Generation}
Next nodes: {H2 Fuel Cells}
Next nodes: set()
Changes in this iteration: 5
Next nodes: {H2 Electrolysis, Nuclear}
Next nodes: {Electricity Generation}
Next nodes: {H2 Fuel Cells}
Next nodes: set()
Changes in this iteration: 0
Converged with 2 iterations.
